In [1]:
#!pip install pyvi
#!pip install stop-words
#!pip install stopwords
#!pip install googletrans
#!pip install google_trans_new
#!pip install deep_translator


In [2]:
from Trans_Tokenize import *
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
from sklearn.model_selection import train_test_split
import math
import unicodedata

In [3]:
raw_data=pd.read_csv('../dataset/job_data.csv')
start = 500  # Vị trí bắt đầu (chỉ số hàng)
end = 650    # Vị trí kết thúc (không bao gồm hàng này)

# Lấy các dòng từ vị trí start đến end
raw_data = raw_data.iloc[start:end]

### Extract dataframe from few features

In [5]:
extracted_data=raw_data[['Company Name','Location','Experience','Job Level','Industry','Employment Type','Welfare','Job Description','Job Requirements','Follower','Company Address','Salary']]

### Drop some duplicate job application. Some company create the same after few day and change a bit 

In [6]:

extracted_data = extracted_data.drop_duplicates(subset=['Job Description','Job Level','Company Name','Location'], keep='first')


### Format Location

In [ ]:

extracted_data['Location'] = extracted_data['Location'].apply(lambda x: x.title())

extracted_data['Location'] = extracted_data['Location'].apply(lambda x: x.split(',')[0].strip() if ',' in x else x.strip())

filtered_location_data = extracted_data[~extracted_data['Location'].isin(['Not Specified', 'Not Found'])]



<class 'pandas.core.frame.DataFrame'>
Index: 147 entries, 500 to 649
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Company Name      147 non-null    object
 1   Location          147 non-null    object
 2   Experience        147 non-null    object
 3   Job Level         147 non-null    object
 4   Industry          147 non-null    object
 5   Employment Type   147 non-null    object
 6   Welfare           147 non-null    object
 7   Job Description   147 non-null    object
 8   Job Requirements  147 non-null    object
 9   Follower          147 non-null    object
 10  Company Address   147 non-null    object
 11  Salary            147 non-null    object
dtypes: object(12)
memory usage: 14.9+ KB


### Experience prepossessing

In [8]:
def extract_all_numbers(value):
    value = value.replace(',', '')
    value = value.replace('.', '')
    numbers = re.findall(r'\d+', value)
    numbers = [int(num) for num in numbers]
    
    if len(numbers) > 1 and (numbers[1] / numbers[0] > 2):
        return numbers[0]
    elif numbers:
        return sum(numbers) / len(numbers)  # Tính trung bình nếu có số
    return 0  
 

In [9]:
filtered_location_data.loc[:, 'Experience'] = filtered_location_data['Experience'].apply(extract_all_numbers)
filtered_experience_data = filtered_location_data

### Industry data

In [11]:

filtered_experience_data.loc[:,'Industry'] = filtered_experience_data['Industry'].apply(translate_column)

In [12]:
filtered_experience_data.loc[:,'Industry']=filtered_experience_data['Industry'].apply(process_tokenization_english)

### Employment Type

In [13]:
filtered_employee_data=filtered_experience_data.copy()
#filtered_employee_data['Employment Type'].value_counts()

In [14]:
filtered_employee_data.loc[:, 'Employment Type'] = filtered_employee_data['Employment Type'].replace({
    'Toàn thời gian': 'Official',
    'Nhân viên chính thức': 'Official',
    'Thời vụ/ Nghề tự do': 'Part-time'
})

### Job Level preprocessing

In [15]:
filtered_job_level_data = filtered_employee_data.copy()
filtered_job_level_data.loc[:, 'Job Level'] = filtered_job_level_data['Job Level'].replace({
    'Nhân viên': 'Employee',
    'Trưởng nhóm / Giám sát': 'Leader',
    'Quản lý': 'Manager',
    'Giám đốc': 'Chief',
})

### Welfare preprocessing

In [16]:
filtered_job_level_data.loc[:,'Welfare'] = filtered_job_level_data['Welfare'].apply(translate_column)

In [17]:
filtered_job_level_data.loc[:,'Welfare']=filtered_job_level_data['Welfare'].apply(process_tokenization_english)

In [21]:
filtered_welfare_data=filtered_job_level_data.copy()    

### Follower preprocessing

In [24]:

filtered_follower_data = filtered_welfare_data.copy()
filtered_follower_data['Follower'] = filtered_follower_data['Follower'].astype(str)

filtered_follower_data['Follower'] = filtered_follower_data['Follower'].replace('nan', '0').replace('Not Found', '0').replace('Not specified', '0')

filtered_follower_data['Follower'] = filtered_follower_data['Follower'].apply(lambda x: int(x.replace('.', '')))


### Job Requirements Preprocessing

In [25]:
filtered_requirement_data=filtered_follower_data.copy()
filtered_requirement_data['Job Requirements'] = filtered_requirement_data['Job Requirements'].replace(
    ['', 'Not Found', 'not found', 'Not specifield', 'Not Specifield'], 
    'No requirements needed')


In [26]:
filtered_requirement_data.loc[:,'Job Requirements'] = filtered_requirement_data['Job Requirements'].apply(translate_column)

In [27]:
filtered_requirement_data.loc[:,'Job Requirements']=filtered_requirement_data['Job Requirements'].apply(process_tokenization_english)

### Salary preprocessing

In [28]:
import re
import math

def extract_salary(value):
    # Loại bỏ dấu phẩy trong số (để xử lý 700,000 => 700000)
    value = value.replace(',', '')
    value = value.replace('.', '')
    # Tách các con số trong chuỗi
    numbers = re.findall(r'\d+', value)
    numbers = [int(num) for num in numbers]  # Chuyển các chuỗi số thành integer
    
    # Nếu có khoảng lương
    if len(numbers) > 1:
        # Nếu khoảng chênh lệch giữa hai giá trị quá lớn (ví dụ: 700 - 1,000 USD)
        if (numbers[1] - numbers[0]) / numbers[0] > 0.5:
            salary = numbers[0] + (numbers[1] - numbers[0]) / 4  # Đoán mức lương trung bình trong khoảng
        else:
            salary = sum(numbers) / len(numbers)  # Tính trung bình nếu khoảng cách hợp lý
    elif numbers:
        # Trường hợp chỉ có một số (mức lương cố định)
        salary = sum(numbers) / len(numbers)
    else:
        # Nếu không có giá trị số (có thể là "Cạnh tranh" hay tương tự)
        return 'Cạnh tranh'
    
    # Kiểm tra các đơn vị tiền tệ và chuyển đổi
    if any(unit in value.lower() for unit in ['triệu', 'tr']):
        salary *= 1_000_000  # Nếu là triệu, nhân với 1,000,000
    elif any(unit in value.lower() for unit in ['usd', 'đô la', '$', 'đô']):
        salary *= 23_000  # Nếu là USD, nhân với tỷ giá (ví dụ: 1 USD = 23,000 VND)

    # Kiểm tra nếu là mức lương theo năm và chuyển sang mức lương tháng
    if any(unit in value.lower() for unit in ['năm', 'year']):
        salary /= 12  # Chia cho 12 để chuyển từ lương năm sang lương tháng

    # Làm tròn kết quả về bội số gần nhất của 100,000
    return math.ceil(round(salary, -5))



In [29]:
filtered_salary_data = filtered_requirement_data.copy()

filtered_salary_data['Salary'] = filtered_salary_data['Salary'].replace(
    ['0','Thỏa thuận', 'Negotiable', 'Not Specified', 'Not Found', 'not found', 'Not specifield'], 
    np.nan)
filtered_salary_data['Salary'] = filtered_salary_data['Salary'].apply(extract_salary)

In [30]:
filtered_salary_data['Salary'].value_counts()

Salary
Cạnh tranh    57
11000000       6
10000000       6
9000000        5
18800000       5
9800000        4
20000000       4
10500000       4
15500000       3
17500000       3
8000000        3
12500000       3
25000000       3
10200000       2
17000000       2
14000000       2
14800000       2
22500000       2
35000000       2
15000000       2
80000000       2
12000000       2
7000000        2
37500000       1
41000000       1
18000000       1
5000000        1
28800000       1
14400000       1
8200000        1
19500000       1
37000000       1
16200000       1
21500000       1
75000000       1
13200000       1
23800000       1
8500000        1
7500000        1
12200000       1
13500000       1
19000000       1
23000000       1
13000000       1
Name: count, dtype: int64

In [31]:
processed_data=filtered_salary_data[['Location','Experience','Job Level','Industry','Employment Type','Welfare','Job Requirements','Follower','Salary']]

### Reduced Token


In [32]:
def remove_common_tokens(row):
    # Đảm bảo cả hai cột là chuỗi trước khi split
    job_requirements_text = ' '.join(row['Job Requirements']) if isinstance(row['Job Requirements'], list) else row['Job Requirements']
    welfare_text = ' '.join(row['Welfare']) if isinstance(row['Welfare'], list) else row['Welfare']
    
    # Tách các từ trong cả hai cột và chuyển thành set
    job_requirements_tokens = set(job_requirements_text.split())
    welfare_tokens = set(welfare_text.split())
    
    # Xóa các từ chung giữa Job Requirements và Welfare
    unique_tokens = job_requirements_tokens - welfare_tokens
    
    # Gộp các từ còn lại thành chuỗi và trả về
    return unique_tokens

In [ ]:

processed_data.loc[:,'Job Requirements'] = processed_data.apply(remove_common_tokens, axis=1)


In [37]:
processed_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 147 entries, 500 to 649
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Location          147 non-null    object
 1   Experience        147 non-null    object
 2   Job Level         147 non-null    object
 3   Industry          147 non-null    object
 4   Employment Type   147 non-null    object
 5   Welfare           147 non-null    object
 6   Job Requirements  147 non-null    object
 7   Follower          147 non-null    int64 
 8   Salary            147 non-null    object
dtypes: int64(1), object(8)
memory usage: 11.5+ KB


### Separate and write file

In [34]:
Defined_salary=processed_data[processed_data['Salary']!='Cạnh tranh']
Undefined_salary=processed_data[processed_data['Salary']=='Cạnh tranh']

In [35]:
Defined_salary.to_csv('../dataset/Defined_salary.csv', index=False,encoding='utf-8-sig')
Undefined_salary.to_csv('../dataset/Undefined_salary.csv', index=False,encoding='utf-8-sig')